HW2 : Gughapriyaa Elango<br>
SU ID : 585992381

In [8]:
#Import pandas
import pandas as pd
#Import numpy
import numpy as np
#Read the given tsv file
data =pd.read_csv("deception_data_converted_final.tsv", delimiter='\t')
data.head(5)

,lie,sentiment,review
0,f,n,"'Mike\'s Pizza High Point, NY Service was very..."
1,f,n,'i really like this buffet restaurant in Marsh...
2,f,n,"'After I went shopping with some of my friend,..."
3,f,n,'Olive Oil Garden was very disappointing. I ex...
4,f,n,'The Seven Heaven restaurant was never known f...


In [9]:
data.tail(5)

,lie,sentiment,review
87,t,p,'Pastablities is a locally owned restaurant in...
88,t,p,'I like the Pizza at Dominoes for their specia...
89,t,p,'It was a really amazing Japanese restaurant. ...
90,t,p,'How do I even pick a best experience at Joe\'...
91,t,p,'My sister and I ate at this restaurant called...


The beginning of the records have lie as false and sentiment as negative. The end of the dataframe have lie as true and sentiment as positive.

In [10]:
#Since the sentiment of each restaurant review are in categorical as "t" and "f"
#And sentiment is in "n" and "p"
labels_sen = data['sentiment'].map({'p': 1, 'n': 0})
labels_lie = data['lie'].map({'t': 1, 'f': 0})

In [11]:
#Lets split the data into train and test before running models - sentiment:
from sklearn.model_selection import train_test_split
X_train_sentiment, X_test_sentiment, y_train_sentiment, y_test_sentiment = train_test_split(
    data['review'], labels_sen, test_size=0.3, random_state=42)

In [12]:
#Lets split the data into train and test before running models - lie:
X_train_lie, X_test_lie, y_train_lie, y_test_lie = train_test_split(
    data['review'], labels_lie, test_size=0.3, random_state=42)

In [14]:
#Vectorize the reviews using different vectorization options:
#CountVectorizer binary, bigram and non binary and TF-IDF - 4 different vectorizers tried.
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vectorizers = {
    'TF-IDF Vectorizer': TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=5, stop_words='english'),
    'CountVectorizer Binary': CountVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english'),
    'CountVectorizer': CountVectorizer(encoding='latin-1', binary=False, min_df=5, stop_words='english'),
    'CountVectorizer Bigram': CountVectorizer(encoding='latin-1', ngram_range=(1, 2), min_df=5, stop_words='english')
}

In [16]:
#To build the model for Multinomial Naive bayes and Support vector machine:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
classifiers = {
    'Multinomial Naive Bayes': MultinomialNB(),
    'Support Vector Machine': SVC(),
}

#Lets create an empty variable to store the results of each model:
results = {}

In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict
#Lets write a for loop to loop through each vectorize and each classifier:
for vectorizer_name, vectorizer in vectorizers.items():
    for classifier_name, classifier in classifiers.items():
        #Loop through the vectorizers above
        X_train_vec = vectorizer.fit_transform(X_train_sentiment)
        #First try sentiment model:
        sentiment_model = classifier.fit(X_train_vec, y_train_sentiment)
        X_test_vec = vectorizer.transform(X_test_sentiment)
        sentiment_preds = sentiment_model.predict(X_test_vec)
        print(f"Vectorization: {vectorizer_name}, Classifier: {classifier_name}")
        #5-fold cross validation for sentiment:
        sentiment_scores = cross_val_score(classifier, X_train_vec, y_train_sentiment, cv=5, scoring='accuracy')
        sentiment_preds_cv = cross_val_predict(classifier, X_train_vec, y_train_sentiment, cv=5)
        #Evaluate sentiment:
        sentiment_accuracy = sentiment_scores.mean()
        sentiment_precision = precision_score(y_train_sentiment, sentiment_preds_cv)
        sentiment_recall = recall_score(y_train_sentiment, sentiment_preds_cv)
        sentiment_f1 = f1_score(y_train_sentiment, sentiment_preds_cv)
        #Print sentiment results:
        results[f'Sentiment_{vectorizer_name}_{classifier_name}'] = {
            'Accuracy': sentiment_accuracy,
            'Precision': sentiment_precision,
            'Recall': sentiment_recall,
            'F1 Score': sentiment_f1,
        }
        print(f"Sentiment Model - Accuracy: {sentiment_accuracy:.2f}, Precision: {sentiment_precision:.2f}, Recall: {sentiment_recall:.2f}, F1 Score: {sentiment_f1:.2f}")
        sentiment_confusion = confusion_matrix(y_test_sentiment, sentiment_preds)
        print("Confusion Matrix for Sentiment Model:")
        print(sentiment_confusion)
        #Print featues:
        feature_names = vectorizer.get_feature_names_out()
        if classifier_name == 'Multinomial Naive Bayes':
            log_probabilities = sentiment_model.feature_log_prob_
            feature_importances = log_probabilities[1] - log_probabilities[0]
        elif classifier_name == 'SVM':
            feature_importances = sentiment_model.coef_.toarray()[0]  # For SVM with linear kernel
        top_10_features = [feature_names[i] for i in feature_importances.argsort()[-10:]][::-1]
        print(f"Vectorization: {vectorizer_name}, Classifier: {classifier_name}, Task: Sentiment")
        print("Top 10 Features:")
        for i, feature in enumerate(top_10_features):
            print(f"{i+1}. {feature}")


        #Second try lie model
        lie_model = classifier.fit(X_train_vec, y_train_lie)
        lie_preds = lie_model.predict(X_test_vec)
        #5-fold cross-validation for lie:
        lie_scores = cross_val_score(classifier, X_train_vec, y_train_lie, cv=5, scoring='accuracy')
        lie_preds_cv = cross_val_predict(classifier, X_train_vec, y_train_lie, cv=5)
        #Evaluate lie:
        lie_accuracy = lie_scores.mean()
        lie_precision = precision_score(y_train_lie, lie_preds_cv)
        lie_recall = recall_score(y_train_lie, lie_preds_cv)
        lie_f1 = f1_score(y_train_lie, lie_preds_cv)

        #Print lie results:
        results[f'lie_{vectorizer_name}_{classifier_name}'] = {
            'Accuracy': lie_accuracy,
            'Precision': lie_precision,
            'Recall': lie_recall,
            'F1 Score': lie_f1,
        }
        print(f"Lie Model - Accuracy: {lie_accuracy:.2f}, Precision: {lie_precision:.2f}, Recall: {lie_recall:.2f}, F1 Score: {lie_f1:.2f}")
        lie_confusion = confusion_matrix(y_test_lie, lie_preds)
        print("Confusion Matrix for Lie Model:")
        print(lie_confusion)

        #Print top features for lie model:
        if classifier_name == 'Multinomial Naive Bayes':
            log_probabilities = lie_model.feature_log_prob_
            feature_importances = log_probabilities[1] - log_probabilities[0]
        elif classifier_name == 'SVM':
            feature_importances = lie_model.coef_.toarray()[0]  # For SVM with linear kernel
        top_10_features = [feature_names[i] for i in feature_importances.argsort()[-10:]][::-1]
        print(f"Vectorization: {vectorizer_name}, Classifier: {classifier_name}, Task: Sentiment")
        print("Top 10 Features:")
        for i, feature in enumerate(top_10_features):
            print(f"{i+1}. {feature}")

        print(f"---End of model---")

Vectorization: TF-IDF Vectorizer, Classifier: Multinomial Naive Bayes
Sentiment Model - Accuracy: 0.79, Precision: 0.74, Recall: 0.94, F1 Score: 0.83
Confusion Matrix for Sentiment Model:
[[10  6]
 [ 0 12]]
Vectorization: TF-IDF Vectorizer, Classifier: Multinomial Naive Bayes, Task: Sentiment
Top 10 Features:
1. great
2. best
3. amazing
4. delicious
5. quality
6. need
7. friendly
8. nice
9. fresh
10. favorite
Lie Model - Accuracy: 0.45, Precision: 0.47, Recall: 0.55, F1 Score: 0.51
Confusion Matrix for Lie Model:
[[7 8]
 [6 7]]
Vectorization: TF-IDF Vectorizer, Classifier: Multinomial Naive Bayes, Task: Sentiment
Top 10 Features:
1. environment
2. people
3. cheese
4. worst
5. waitress
6. menu
7. lunch
8. good
9. overall
10. time
---End of model---
Vectorization: TF-IDF Vectorizer, Classifier: Support Vector Machine
Sentiment Model - Accuracy: 0.78, Precision: 0.76, Recall: 0.85, F1 Score: 0.81
Confusion Matrix for Sentiment Model:
[[11  5]
 [ 0 12]]
Vectorization: TF-IDF Vectorizer, Cl